# Initialize Section

## Import Section

In [ ]:
from datetime import datetime
from google.colab import drive
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchsummary import summary
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
%matplotlib inline
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import itertools
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, normalized_mutual_info_score, adjusted_rand_score, f1_score

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import tqdm
import os.path as osp

import torch
import matplotlib

matplotlib.use("Agg")
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col

import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as col

from torch.utils.data import TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import SGD
import numpy as np
import os

import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
# from .meter import AverageMeter
# from ..metric import binary_accuracy


import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from google.colab import drive

from torch.utils.data import DataLoader, WeightedRandomSampler
import itertools
from torch.autograd import Function
from torch.autograd import Function

import os
import random
from glob import glob
import zipfile
import os

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
!pip install torchmetrics
import torchmetrics


import os

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torchmetrics import Accuracy, ConfusionMatrix, F1Score, Recall

import os
import time
import random

import torch.backends.cudnn as cudnn
import torch.utils.data

from torchvision import datasets, transforms
from torchmetrics.classification import F1Score

import os
import sys
import random
import numpy as np
import argparse
import psutil

import torch
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader, WeightedRandomSampler
import itertools
import json

import torch
import os
import csv
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


from torch.utils.data import TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import SGD

from typing import Optional, List

import torch
import os
import csv
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from pathlib import Path

import torch.nn.utils.prune as prune

## Global Parameter

In [ ]:
image_size = 224
batch_size = 64
channel_size = 3
lr = 1e-3
num_classes = 31
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.manual_seed(123)

## Used Function

In [ ]:
# Plot Graph Function
def plot_graph(history, name):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(10)
    fig.suptitle(f"Train vs Validation ({name})")
    ax1.plot(history["train_student_acc"], label="Train")
    ax1.plot(history["val_student_acc"], label="Validation")
    ax1.legend()
    ax1.set_title("Accuracy")

    ax2.plot(history["train_student_loss"], label="Train")
    ax2.plot(history["val_student_loss"], label="Validation")
    ax2.legend()
    ax2.set_title("Loss")
    fig.show()


def print_model_param_nums(model=None):
    if model == None:
        model = torchvision.models.alexnet()
    total = sum([param.nelement() if param.requires_grad else 0 for param in model.parameters()])
    print('  + Number of params: %.4fM' % (total / 1e6))

def count_model_param_flops(model=None, input_res=224, multiply_adds=True):

    prods = {}
    def save_hook(name):
        def hook_per(self, input, output):
            prods[name] = np.prod(input[0].shape)
        return hook_per

    list_1=[]
    def simple_hook(self, input, output):
        list_1.append(np.prod(input[0].shape))
    list_2={}
    def simple_hook2(self, input, output):
        list_2['names'] = np.prod(input[0].shape)


    list_conv=[]
    def conv_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups)
        bias_ops = 1 if self.bias is not None else 0

        params = output_channels * (kernel_ops + bias_ops)
        # flops = (kernel_ops * (2 if multiply_adds else 1) + bias_ops) * output_channels * output_height * output_width * batch_size

        num_weight_params = (self.weight.data != 0).float().sum()
        flops = (num_weight_params * (2 if multiply_adds else 1) + bias_ops * output_channels) * output_height * output_width * batch_size

        list_conv.append(flops)

    list_linear=[]
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1

        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()

        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)

    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = 0
        flops = (kernel_ops + bias_ops) * output_channels * output_height * output_width * batch_size

        list_pooling.append(flops)

    list_upsample=[]

    # For bilinear upsample
    def upsample_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        flops = output_height * output_width * output_channels * batch_size * 12
        list_upsample.append(flops)

    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            if isinstance(net, torch.nn.Upsample):
                net.register_forward_hook(upsample_hook)
            return
        for c in childrens:
            foo(c)

    if model == None:
        model = torchvision.models.alexnet()
    foo(model)
    input = Variable(torch.rand(3,input_res,input_res).unsqueeze(0), requires_grad = True).to(device)
    out = model(input)


    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling) + sum(list_upsample))

    print('Number of FLOPs: %.6f GFLOPs (%.2f MFLOPs)' % (total_flops / 1e9, total_flops / 1e6))

    return total_flops

# Inference Function (Collect lb, loss)
def inference(loaded_model, validation_loader, mode="cuda"):
    device = torch.device(mode)
    if mode == "cpu":
        loaded_model.cpu()
        stored_lbs, stored_preds = torch.empty(0, dtype=torch.float32), torch.empty(0, dtype=torch.float32)
    else:
        loaded_model.to(device)
        stored_lbs, stored_preds = torch.empty(0, dtype=torch.float32).to(device), torch.empty(0, dtype=torch.float32).to(device)
    acc_test = 0
    test_loss = 0
    loss_fn = torch.nn.CrossEntropyLoss()
    loaded_model.eval()

    for i, tdata in enumerate(validation_loader):
        with torch.no_grad():
            tinputs, tlabels = tdata[0].to(device), tdata[1].to(device)
            toutputs = loaded_model(tinputs)
            if isinstance(toutputs, (tuple, list)):
                toutputs = toutputs[0]
            loss = loss_fn(toutputs, tlabels)
            test_loss += loss
            _, preds_t = torch.max(toutputs, 1)
            acc_test += (preds_t == tlabels).float().mean().item()
            stored_lbs = torch.cat((stored_lbs, tlabels), 0)
            stored_preds = torch.cat((stored_preds, preds_t), 0)

    accuracy_t = round(acc_test / float(len(validation_loader)), 4)
    avg_tloss = test_loss / (i + 1)
    print(f"[(Inference || test loss: {avg_tloss}] [accuracy_test: {accuracy_t * 100} %]")

    return stored_lbs, stored_preds, avg_tloss.item(), accuracy_t




def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Blues, show_labels=False, show_numbers=False):
    """
    This function prints and plots the confusion matrix.
    given a sklearn confusion matrix (cm), make a nice plot.

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
    classes:      class names, for example: ['class1', 'class2', ...]
    title:        the text to display at the top of the matrix
    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm

    normalize:    If False, plot the raw numbers.
                  If True, plot the proportions.
    show_labels:  If False, hide class labels on the x and y axes.
    show_numbers: If False, hide the numbers inside each grid.
    """
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if show_labels:
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45, fontsize=10)
        plt.yticks(tick_marks, classes, fontsize=10)
    else:
        plt.xticks([])
        plt.yticks([])

    if normalize:
        cm = cm.astype("float") / (cm.sum(axis=1)[:, np.newaxis] + 1)

    if show_numbers:
        formated = ".2f" if normalize else "d"
        thresh = cm.max() / 2.0
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], formated), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.gcf().set_size_inches(10, 8)
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()



def collect_feature(data_loader: DataLoader, feature_extractor: nn.Module, device: torch.device, max_num_features=None) -> torch.Tensor:
    feature_extractor.eval()
    all_features = []
    all_labels = []
    with torch.no_grad():
        for i, (images, target) in enumerate(tqdm.tqdm(data_loader)):
            images = images.to(device)
            features = feature_extractor(images)
            if isinstance(features, tuple):
                feature_tensor = features[0]
                feature_tensor = feature_tensor.to(device)
            else:
                feature_tensor = features.to(device)

            all_features.append(feature_tensor)
            all_labels.append(target)

    return torch.cat(all_features, dim=0), torch.cat(all_labels, dim=0)




def visualize(source_feature: torch.Tensor, target_feature: torch.Tensor, filename: str, source_color="r", target_color="b"):
    """
    Visualize features from different domains using t-SNE.
    Args:
        source_feature (tensor): features from source domain in shape :math:`(minibatch, F)`
        target_feature (tensor): features from target domain in shape :math:`(minibatch, F)`
        filename (str): the file name to save t-SNE
        source_color (str): the color of the source features. Default: 'r'
        target_color (str): the color of the target features. Default: 'b'
    """
    source_feature = source_feature.cpu().numpy()
    target_feature = target_feature.cpu().numpy()
    features = np.concatenate([source_feature, target_feature], axis=0)

    X_tsne = TSNE(n_components=2, random_state=33).fit_transform(features)

    domains = np.concatenate((np.ones(len(source_feature)), np.zeros(len(target_feature))))

    plt.figure(figsize=(10, 10))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=domains, cmap=col.ListedColormap([source_color, target_color]), s=20)  # default: s=2
    plt.savefig(filename)



"""
  Arg: t-SNE for class clustering visualization
"""


def visualize_class_n_domain(
    source_feature: torch.Tensor, target_feature: torch.Tensor, source_labels: torch.Tensor, target_labels: torch.Tensor, filename: str, source_color="r", target_color="b"
):
    """
    Visualize features from different domains using t-SNE.
    Args:
        source_feature (tensor): features from source domain in shape :math:`(minibatch, F)`
        target_feature (tensor): features from target domain in shape :math:`(minibatch, F)`
        source_labels (tensor): class labels for source domain features
        target_labels (tensor): class labels for target domain features
        filename (str): the file name to save t-SNE
        source_color (str): the color of the source features. Default: 'r'
        target_color (str): the color of the target features. Default: 'b'
    """
    source_feature = source_feature.cpu().numpy()
    target_feature = target_feature.cpu().numpy()
    source_labels = source_labels.cpu().numpy()
    target_labels = target_labels.cpu().numpy()

    features = np.concatenate([source_feature, target_feature], axis=0)
    labels = np.concatenate([source_labels, target_labels], axis=0)
    domains = np.concatenate((np.ones(len(source_feature)), np.zeros(len(target_feature))))

    X_tsne = TSNE(n_components=2, random_state=33).fit_transform(features)

    plt.figure(figsize=(10, 10))

    unique_labels = np.unique(labels)

    cmap_s = plt.get_cmap("nipy_spectral", len(unique_labels))
    cmap_r = plt.get_cmap("gist_rainbow", len(unique_labels))

    for label in unique_labels:
        for domain in [0, 1]:
            mask = (labels == label) & (domains == domain)
            plt.scatter(
                X_tsne[mask, 0],
                X_tsne[mask, 1],
                c=cmap_s(label),
                s=10,
                # label=f"Class {label}, Domain {domain}",
            )

    plt.legend()
    plt.savefig(filename)


"""
  Args: A-distance computation
  Ref: https://github.com/zhjscut/Bridging_UDA_SSL/blob/e0be6742f1203bb983261e3e1e57d34e1e03299d/common/utils/analysis/a_distance.py
"""

def binary_accuracy(output: torch.Tensor, target: torch.Tensor) -> float:
    """Computes the accuracy for binary classification"""
    with torch.no_grad():
        batch_size = target.size(0)
        pred = (output >= 0.5).float().t().view(-1)
        correct = pred.eq(target.view(-1)).float().sum()
        correct.mul_(100. / batch_size)
        return correct

class ANet(nn.Module):
    def __init__(self, in_feature):
        super(ANet, self).__init__()
        self.layer = nn.Linear(in_feature, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer(x)
        x = self.sigmoid(x)
        return x


def Adist_calculate(source_feature: torch.Tensor, target_feature: torch.Tensor,
              device, progress=True, training_epochs=10):
    """
    Calculate the :math:`\mathcal{A}`-distance, which is a measure for distribution discrepancy.
    The definition is :math:`dist_\mathcal{A} = 2 (1-2\epsilon)`, where :math:`\epsilon` is the
    test error of a classifier trained to discriminate the source from the target.
    Args:
        source_feature (tensor): features from source domain in shape :math:`(minibatch, F)`
        target_feature (tensor): features from target domain in shape :math:`(minibatch, F)`
        device (torch.device)
        progress (bool): if True, displays a the progress of training A-Net
        training_epochs (int): the number of epochs when training the classifier
    Returns:
        :math:`\mathcal{A}`-distance
    """
    source_label = torch.ones((source_feature.shape[0], 1))
    target_label = torch.zeros((target_feature.shape[0], 1))
    feature = torch.cat([source_feature, target_feature], dim=0)
    label = torch.cat([source_label, target_label], dim=0)

    dataset = TensorDataset(feature, label)
    length = len(dataset)
    train_size = int(0.8 * length)
    val_size = length - train_size

    # Specify a random generator for CUDA
    generator = torch.Generator(device=device)

    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size], generator)
    train_loader = DataLoader(train_set, batch_size=2, shuffle=True, generator=generator)
    val_loader = DataLoader(val_set, batch_size=8, shuffle=False, generator=generator)

    anet = ANet(feature.shape[1]).to(device)
    optimizer = SGD(anet.parameters(), lr=0.01)
    a_distance = 2.0
    for epoch in range(training_epochs):
        anet.train()
        for (x, label) in train_loader:
            x = x.to(device)
            label = label.to(device)
            anet.zero_grad()
            y = anet(x)
            loss = F.binary_cross_entropy(y, label)
            loss.backward()
            optimizer.step()

        anet.eval()
        meter = AverageMeter("accuracy", ":4.2f")
        with torch.no_grad():
            for (x, label) in val_loader:
                x = x.to(device)
                label = label.to(device)
                y = anet(x)
                acc = binary_accuracy(y, label)
                meter.update(acc, x.shape[0])
        error = 1 - meter.avg / 100
        a_distance = 2 * (1 - 2 * error)
        if progress:
            print("epoch {} accuracy: {} A-dist: {}".format(epoch, meter.avg, a_distance))

    return a_distance




class AverageMeter(object):
    r"""Computes and stores the average and current value.
    Examples::
        >>> # Initialize a meter to record loss
        >>> losses = AverageMeter()
        >>> # Update meter after every minibatch update
        >>> losses.update(loss_value, batch_size)
    """
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class AverageMeterDict(object):
    def __init__(self, names: List, fmt: Optional[str] = ':f'):
        self.dict = {
            name: AverageMeter(name, fmt) for name in names
        }

    def reset(self):
        for meter in self.dict.values():
            meter.reset()

    def update(self, accuracies, n=1):
        for name, acc in accuracies.items():
            self.dict[name].update(acc, n)

    def average(self):
        return {
            name: meter.avg for name, meter in self.dict.items()
        }

    def __getitem__(self, item):
        return self.dict[item]


class Meter(object):
    """Computes and stores the current value."""
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0

    def update(self, val):
        self.val = val

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


In [ ]:
def plot_graphDA(history, name):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(10)
    fig.suptitle(f"Train vs Validation {name}")
    ax1.plot(history["train_src_acc"], label="Train_src_acc")
    ax1.plot(history["train_tar_acc"], label="Train_tar_acc")
    ax1.legend()
    ax1.set_title("Accuracy (Src/Tar)")

    ax2.plot(history["train_loss"], label="Train_loss")
    ax2.legend()
    ax2.set_title("Loss")
    fig.show()

In [ ]:
def get_loader_DA(src, tar):
  if src == "amazon":
    src_tl = amazon_tl
  elif src == "dslr":
    src_tl = dslr_tl
  elif src == "webcam":
    src_tl = webcam_tl
  else:
    print("Undefined Source")
    return None, None
  if tar == "amazon":
    tar_tl = amazon_tl
  elif tar == "dslr":
    tar_tl = dslr_tl
  elif tar == "webcam":
    tar_tl = webcam_tl
  else:
    print("Undefined Target")
    return None, None
  return src_tl, tar_tl

def get_loader(sd):

    if sd == "amazon":
        return amazon_tl, amazon_vl
    elif sd == "dslr":
        return dslr_tl, dslr_vl
    elif sd == "webcam":
        return webcam_tl, webcam_vl
    else:
        print("Undefined Dataset")
        return None, None

In [ ]:
def save_training_logs(logs, filename):
    with open(filename, 'w') as f:
        json.dump(logs, f, indent=4)
    print(f"Training logs saved to {filename}")

def load_training_logs(filename):
    with open(filename, 'r') as f:
        logs = json.load(f)
    print(f"Training logs loaded from {filename}")
    return logs

In [ ]:
def zip_directory(folder_path, zip_name):
    # Create a zip file
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Walk through the directory and add files
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

def zip_log_files(zip_name='log_files.zip'):
    """
    Zips all files in the current directory that start with 'LOG_'.

    Parameters:
    zip_name (str): The name of the resulting zip file.
    """
    # Create a Zip file
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Loop through files in the current directory
        for file in os.listdir('.'):
            # Check if the file starts with 'LOG_'
            if file.startswith('LOG_'):
                zipf.write(file)
                print(f"Added {file} to the zip")

    print(f"All files starting with 'LOG_' have been zipped into {zip_name}")

In [ ]:
# @title FLOPS computation
# Code from https://github.com/Eric-mingjie/rethinking-network-pruning/blob/master/imagenet/l1-norm-pruning/compute_flops.py
import numpy as np
import os

import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable


def print_model_param_nums(model=None):
    if model == None:
        model = torchvision.models.alexnet()
    total = sum([param.nelement() if param.requires_grad else 0 for param in model.parameters()])
    print('  + Number of params: %.4fM' % (total / 1e6))

def count_model_param_flops(model=None, input_res=224, multiply_adds=True):

    prods = {}
    def save_hook(name):
        def hook_per(self, input, output):
            prods[name] = np.prod(input[0].shape)
        return hook_per

    list_1=[]
    def simple_hook(self, input, output):
        list_1.append(np.prod(input[0].shape))
    list_2={}
    def simple_hook2(self, input, output):
        list_2['names'] = np.prod(input[0].shape)


    list_conv=[]
    def conv_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups)
        bias_ops = 1 if self.bias is not None else 0

        params = output_channels * (kernel_ops + bias_ops)
        # flops = (kernel_ops * (2 if multiply_adds else 1) + bias_ops) * output_channels * output_height * output_width * batch_size

        num_weight_params = (self.weight.data != 0).float().sum()
        flops = (num_weight_params * (2 if multiply_adds else 1) + bias_ops * output_channels) * output_height * output_width * batch_size

        list_conv.append(flops)

    list_linear=[]
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1

        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()

        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)

    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = 0
        flops = (kernel_ops + bias_ops) * output_channels * output_height * output_width * batch_size

        list_pooling.append(flops)

    list_upsample=[]

    # For bilinear upsample
    def upsample_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        flops = output_height * output_width * output_channels * batch_size * 12
        list_upsample.append(flops)

    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            if isinstance(net, torch.nn.Upsample):
                net.register_forward_hook(upsample_hook)
            return
        for c in childrens:
            foo(c)

    if model == None:
        model = torchvision.models.alexnet()
    foo(model)
    input = Variable(torch.rand(3,input_res,input_res).unsqueeze(0), requires_grad = True).to(device)
    out = model(input)


    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling) + sum(list_upsample))

    print('Number of FLOPs: %.6f GFLOPs (%.2f MFLOPs)' % (total_flops / 1e9, total_flops / 1e6))

    return total_flops

## Data Proprocessing

In [ ]:
transform = transforms.Compose(
        [transforms.Resize((image_size, image_size)),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

drive.mount('./drive/')
!unzip -o './drive/MyDrive/99H_datasets/Office-31.zip'

dataset_path = './Office-31'
!ls
amazon_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'amazon'), transform=transform)
dslr_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'dslr'), transform=transform)
webcam_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'webcam'), transform=transform)

def split_dataset(dataset, train_ratio=0.75):
    train_size = int(train_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

amazon_train, amazon_test = split_dataset(amazon_dataset)
dslr_train, dslr_test = split_dataset(dslr_dataset)
webcam_train, webcam_test = split_dataset(webcam_dataset)

amazon_loader = []
dslr_loader = []
webcam_loader =[]

amazon_tl = DataLoader(amazon_train, batch_size=batch_size, shuffle=True)
amazon_vl = DataLoader(amazon_test, batch_size=batch_size, shuffle=False)

dslr_tl = DataLoader(dslr_train, batch_size=batch_size, shuffle=True)
dslr_vl = DataLoader(dslr_test, batch_size=batch_size, shuffle=False)

webcam_tl = DataLoader(webcam_train, batch_size=batch_size, shuffle=True)
webcam_vl = DataLoader(webcam_test, batch_size=batch_size, shuffle=False)

amazon_loader.append(amazon_tl)
amazon_loader.append(amazon_vl)

dslr_loader.append(dslr_tl)
dslr_loader.append(dslr_vl)

webcam_loader.append(webcam_tl)
webcam_loader.append(webcam_vl)



print(f'Amazon - Train: {len(amazon_train)}, Test: {len(amazon_test)}')
print(f'DSLR - Train: {len(dslr_train)}, Test: {len(dslr_test)}')
print(f'Webcam - Train: {len(webcam_train)}, Test: {len(webcam_test)}')

def switch_ds(ds):
  training_loader = ds[0]
  validation_loader = ds[1]

training_loader = amazon_loader[0]
validation_loader = amazon_loader[1]

switch_ds(amazon_loader)

Drive already mounted at ./drive/; to attempt to forcibly remount, call drive.mount("./drive/", force_remount=True).
Archive:  ./drive/MyDrive/99H_datasets/Office-31.zip
  inflating: Office-31/amazon/back_pack/frame_0001.jpg  
  inflating: Office-31/amazon/back_pack/frame_0002.jpg  
  inflating: Office-31/amazon/back_pack/frame_0003.jpg  
  inflating: Office-31/amazon/back_pack/frame_0004.jpg  
  inflating: Office-31/amazon/back_pack/frame_0005.jpg  
  inflating: Office-31/amazon/back_pack/frame_0006.jpg  
  inflating: Office-31/amazon/back_pack/frame_0007.jpg  
  inflating: Office-31/amazon/back_pack/frame_0008.jpg  
  inflating: Office-31/amazon/back_pack/frame_0009.jpg  
  inflating: Office-31/amazon/back_pack/frame_0010.jpg  
  inflating: Office-31/amazon/back_pack/frame_0011.jpg  
  inflating: Office-31/amazon/back_pack/frame_0012.jpg  
  inflating: Office-31/amazon/back_pack/frame_0013.jpg  
  inflating: Office-31/amazon/back_pack/frame_0014.jpg  
  inflating: Office-31/amazon/ba

# Model Declaration

## MobileNetV3-Small with 31 class head

In [ ]:
def get_model_SO():
    model = models.mobilenet_v3_small(pretrained=True)
    print("Original Classifier:", model.classifier)
    model.classifier[-1] = nn.Linear(in_features=1024, out_features=31, bias=True)
    print("Mod Classifier:", model.classifier)
    model.to(device)
    return model

In [ ]:
get_model_SO()

Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Mod Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=31, bias=True)
)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

## MobileNetV3-Small w/ DANN (Teacher Ver.)

In [ ]:
class GRL(Function):        #Gradient Reversal Layer Function
    @staticmethod
    def forward(ctx, x, lamda):
        # store context for backprop
        ctx.lamda = lamda

        # forward pass is a no-op
        return x

    @staticmethod
    # upstream end and result the loss then send back to backward pass
    def backward(ctx, grad_output):
        # grad_output is dL/dx (since our forward's output was x)

        # Backward pass is just to -lamda the gradient
        # This will become the dL/dx in the rest of the network
        output = -ctx.lamda * grad_output

        # Must return number of inputs to forward()
        return output, None

class DANNMBNv3s(nn.Module):
    def __init__(self, n_C=31):
        super(DANNMBNv3s, self).__init__()

        self.feature_extractor = models.mobilenet_v3_small(pretrained=True)
        self.feature_extractor.classifier = nn.Identity()

        self.num_cnn_features = self._get_num_features()

        # Label classification (blue section)
        self.class_classifier = nn.Sequential(
            nn.Linear(self.num_cnn_features, 1024),
            nn.Hardswish(inplace=True),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1024, 31),
            nn.LogSoftmax(dim=1),
        )
        self.domain_classifier = nn.Sequential(
            nn.Linear(self.num_cnn_features, 1024),
            nn.Hardswish(inplace=True),
            nn.Linear(1024, 2),
            nn.LogSoftmax(dim=1),
        )


    def forward(self, x, grl_lambda=1.0):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
        elif x.shape[1] != 3:
            raise ValueError("Input images must have 3 channels")

        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        reverse_features = GRL.apply(features, grl_lambda)
        domain_prediction = self.domain_classifier(reverse_features)
        class_prediction = self.class_classifier(features)
        return class_prediction, domain_prediction
    def _get_num_features(self):
        """Helper method to calculate the number of output features from the feature extractor."""
        dummy_input = torch.randn(1, 3, 224, 224)  # Assuming input size is 224x224
        features = self.feature_extractor(dummy_input)
        return features.view(1, -1).size(1)

# Model Training

## Source Only

### Traing Loop Function

In [ ]:
def create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size, blur_sigma=2):
    src_size = len(src_dataloader.dataset)
    tar_size = len(tar_dataloader.dataset)

    # Define a blur transformation
    blur_transform = transforms.GaussianBlur(kernel_size=(5, 5), sigma=blur_sigma)

    def blur_images(batch):
        images, labels = batch
        # Apply blur to images
        blurred_images = blur_transform(images)
        return blurred_images, labels

    # Ensure iterators are reset appropriately for smaller dataset
    if src_size > tar_size:
        Dl_source_iter = iter(src_dataloader)
        Dl_target_iter = itertools.cycle(tar_dataloader)  # Cycle the smaller dataset
        max_batches = len(src_dataloader)

        # Apply blur to the cycled target images
        Dl_target_iter = (blur_images(batch) for batch in Dl_target_iter)
    else:
        Dl_source_iter = itertools.cycle(src_dataloader)  # Cycle the smaller dataset
        Dl_target_iter = iter(tar_dataloader)
        max_batches = len(tar_dataloader)

        # Apply blur to the cycled source images
        Dl_source_iter = (blur_images(batch) for batch in Dl_source_iter)

    return Dl_source_iter, Dl_target_iter, max_batches


def train_model_with_balanced_batches(model, optimizer, loss_fn_class, src, tar, device, num_epochs=30, batch_size=64):
    training_logs_so = {"train_loss": [],  "train_src_acc": [], "train_tar_acc": []}

    src_dataloader, tar_dataloader = get_loader_DA(src, tar)
    model_save_path = f"SO_{src}_to_{tar}.pth"
    best_vloss = 1_000_000.
    for epoch_idx in range(num_epochs):
        train_loss, train_src_correct, train_tar_correct = 0, 0, 0
        actual_src_samples, actual_tar_samples = 0, 0  # Track the actual number of processed samples
        print(f'Epoch {epoch_idx+1:04d} / {num_epochs:04d}', end='\n============\n')

        Dl_source_iter, Dl_target_iter, max_batches = create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size)

        for batch_idx in range(max_batches):
            # Get next batch from source and target
            X_s, y_s = next(Dl_source_iter)
            X_t, y_t = next(Dl_target_iter)

            # Ensure the batches have the same size
            if X_s.shape[0] != X_t.shape[0]:
                min_bs = min(X_s.shape[0], X_t.shape[0])
                X_s, y_s = X_s[:min_bs], y_s[:min_bs]
                X_t, y_t = X_t[:min_bs], y_t[:min_bs]

            # Track the number of samples processed
            actual_src_samples += y_s.size(0)
            actual_tar_samples += y_t.size(0)

            # Send data to device (GPU or CPU)
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            class_prediction_s = model(X_s)
            loss = loss_fn_class(class_prediction_s, y_s)  # Calculate loss on source data

            # Backpropagation
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Calculate source and target accuracy
            with torch.no_grad():
                class_prediction_t = model(X_t)
                train_src_correct += (class_prediction_s.argmax(1) == y_s).float().sum().item()
                train_tar_correct += (class_prediction_t.argmax(1) == y_t).float().sum().item()

            train_loss += loss.item()

            # Print batch info
            print(f'[{batch_idx+1}/{max_batches}] class loss: {loss.item():.4f}')

        # Calculate and store metrics for the epoch
        training_logs_so["train_loss"].append(train_loss / max_batches)
        training_logs_so["train_src_acc"].append(train_src_correct / actual_src_samples)  # Use actual number of samples
        training_logs_so["train_tar_acc"].append(train_tar_correct / actual_tar_samples)  # Use actual number of samples

        print(f'Epoch: {epoch_idx+1} || '
              f'Train_src_acc: {train_src_correct / actual_src_samples:.4f}, '  # Display as fraction
              f'Train_tar_acc: {train_tar_correct / actual_tar_samples:.4f}, '
              f'Train_loss: {train_loss / max_batches:.4f}')
        if train_loss < best_vloss:
            best_vloss = train_loss
            path_save_cp = './cp/'
            if not os.path.exists(path_save_cp): os.mkdir(path_save_cp)
            torch.save(model.state_dict(), path_save_cp+model_save_path)


    save_training_logs(training_logs_so, f"LOG_SO_{src}_to_{tar}")
    return training_logs_so

### Training

In [ ]:

summary(get_model_SO(), input_size=(channel_size, image_size, image_size))
num_epochs = 50
lr = 1e-3
loss_fn_class = torch.nn.CrossEntropyLoss()
srcs = ["amazon", "dslr", "webcam"]
tars = ["amazon", "dslr", "webcam"]
for src in srcs:
    for tar in tars:
        if src != tar:
            model = get_model_SO()
            src_dataloader, tar_dataloader = get_loader_DA(src, tar)
            optimizer = optim.Adam(model.parameters(), lr)
            steps_per_epoch = len(src_dataloader)
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=lr,  # Max learning rate during the cycle
                steps_per_epoch=steps_per_epoch,
                epochs=num_epochs,
                anneal_strategy='linear'  # Can also try 'cos' for cosine annealing
            )
            plot_graphDA(train_model_with_balanced_batches(model, optimizer, loss_fn_class, src, tar, device, num_epochs=num_epochs), f"SO {src} -> {tar}")
        else: print("Source and Target are the same. Skiping...")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Mod Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=31, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 16, 56, 56]             144
       BatchNorm2d-5           [-1, 16, 56, 56]              32
              ReLU-6           [-1, 16, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
    

KeyboardInterrupt: 

In [ ]:
zip_directory('./cp', 'SO_Model.zip')

In [ ]:
zip_log_files("SO_LOG.zip")

Added LOG_SO_amazon_to_webcam to the zip
Added LOG_SO_webcam_to_dslr to the zip
Added LOG_SO_dslr_to_webcam to the zip
Added LOG_SO_amazon_to_dslr to the zip
Added LOG_SO_webcam_to_amazon to the zip
Added LOG_SO_dslr_to_amazon to the zip
All files starting with 'LOG_' have been zipped into SO_LOG.zip


## DANN (Teacher Ver.)

### Training Loop Function

In [ ]:


def create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size, blur_sigma=2):
    src_size = len(src_dataloader.dataset)
    tar_size = len(tar_dataloader.dataset)

    # Define a blur transformation
    blur_transform = transforms.GaussianBlur(kernel_size=(5, 5), sigma=blur_sigma)

    def blur_images(batch):
        images, labels = batch
        # Apply blur to images
        blurred_images = blur_transform(images)
        return blurred_images, labels

    # Ensure iterators are reset appropriately for smaller dataset
    if src_size > tar_size:
        Dl_source_iter = iter(src_dataloader)
        Dl_target_iter = itertools.cycle(tar_dataloader)  # Cycle the smaller dataset
        max_batches = len(src_dataloader)

        # Apply blur to the cycled target images
        Dl_target_iter = (blur_images(batch) for batch in Dl_target_iter)
    else:
        Dl_source_iter = itertools.cycle(src_dataloader)  # Cycle the smaller dataset
        Dl_target_iter = iter(tar_dataloader)
        max_batches = len(tar_dataloader)

        # Apply blur to the cycled source images
        Dl_source_iter = (blur_images(batch) for batch in Dl_source_iter)

    return Dl_source_iter, Dl_target_iter, max_batches



def train_model_with_balanced_batches_DANN(model, optimizer, scheduler, loss_fn_class, loss_fn_domain,
                                           src, tar, device,
                                           num_epochs=30, batch_size=64):
    """
    Train the DANN model using balanced batches from both source and target domain dataloaders.
    Args:
        model: The DANN model (including feature extractor and domain classifier).
        optimizer: Optimizer for the model.
        loss_fn_class: Loss function for classification (e.g., cross-entropy).
        loss_fn_domain: Loss function for domain classification (e.g., binary cross-entropy).
        src_dataloader: DataLoader for the source domain.
        tar_dataloader: DataLoader for the target domain.
        device: Device to run the training on (e.g., 'cuda' or 'cpu').
        num_epochs: Number of training epochs.
        batch_size: Batch size used in training.

    Returns:
        training_logs_so: A dictionary with training loss and accuracies for source and target domains.
    """

    # Logs to track training performance
    training_logs_so = {"train_loss": [], "train_src_acc": [], "train_tar_acc": []}
    src_dataloader, tar_dataloader = get_loader_DA(src, tar)
    model_save_path = f"DANN(Teacher)_{src}_to_{tar}.pth"
    best_vloss = 1_000_000.

    for epoch_idx in range(num_epochs):
        train_loss, train_src_correct, train_tar_correct = 0, 0, 0
        actual_src_samples, actual_tar_samples = 0, 0  # Track the actual number of processed samples

        print(f'Epoch {epoch_idx+1:04d} / {num_epochs:04d}', end='\n============\n')

        # Create balanced iterators for source and target domain
        Dl_source_iter, Dl_target_iter, max_batches = create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size)

        for batch_idx in range(max_batches):
            optimizer.zero_grad()

            # Get next batch from source and target datasets
            X_s, y_s = next(Dl_source_iter)
            X_t, y_t = next(Dl_target_iter)

            # Ensure the batches have the same size by taking the minimum batch size
            if X_s.shape[0] != X_t.shape[0]:
                min_bs = min(X_s.shape[0], X_t.shape[0])
                X_s, y_s = X_s[:min_bs], y_s[:min_bs]
                X_t, y_t = X_t[:min_bs], y_t[:min_bs]

            # Track the number of samples processed
            actual_src_samples += y_s.size(0)
            actual_tar_samples += y_t.size(0)

            # Send data to device (GPU or CPU)
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            # Dynamic adjustment of grl_lambda
            p = float(batch_idx + epoch_idx * max_batches) / (num_epochs * max_batches)
            grl_lambda = (2 / (1 + np.exp(-10 * p)) - 1)

            # Prepare labels for domain classification (source domain = 0, target domain = 1)
            y_s_domain = torch.zeros(X_s.size(0), dtype=torch.long).to(device)
            y_t_domain = torch.ones(X_t.size(0), dtype=torch.long).to(device)

            # Forward pass through the model
            class_prediction_s, domain_prediction_s = model(X_s, grl_lambda)
            _, domain_prediction_t = model(X_t, grl_lambda)

            # Calculate classification and domain losses
            loss_s_label = loss_fn_class(class_prediction_s, y_s)  # Source classification loss
            loss_s_domain = loss_fn_domain(domain_prediction_s, y_s_domain)  # Source domain loss
            loss_t_domain = loss_fn_domain(domain_prediction_t, y_t_domain)  # Target domain loss

            # Combine the losses with a weight for domain loss
            loss = loss_s_label + grl_lambda * (loss_s_domain + loss_t_domain)

            # Backpropagation
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Accumulate the training loss
            train_loss += loss.item()

            # Compute accuracy for source domain classification
            train_src_correct += (class_prediction_s.argmax(1) == y_s).float().sum().item()

            # Compute accuracy for target domain classification (even though target labels are not used in training)
            with torch.no_grad():
                class_prediction_t, _ = model(X_t, grl_lambda)
            train_tar_correct += (class_prediction_t.argmax(1) == y_t).float().sum().item()

            # Print batch info
            print(f'[{batch_idx+1}/{max_batches}] Loss: {loss.item():.4f}')

        # Calculate and store metrics for the epoch
        training_logs_so["train_loss"].append(train_loss / max_batches)
        training_logs_so["train_src_acc"].append(train_src_correct / actual_src_samples)
        training_logs_so["train_tar_acc"].append(train_tar_correct / actual_tar_samples)

        print(f'Epoch: {epoch_idx+1} | '
              f'Source Accuracy: {train_src_correct / actual_src_samples:.4f}, '
              f'Target Accuracy: {train_tar_correct / actual_tar_samples:.4f}, '
              f'Loss: {train_loss / max_batches:.4f}')
        if train_loss < best_vloss:
            best_vloss = train_loss
            path_save_cp = './cp/'
            if not os.path.exists(path_save_cp): os.mkdir(path_save_cp)
            torch.save(model.state_dict(), path_save_cp+model_save_path)


    save_training_logs(training_logs_so, f"LOG_DANN_{src}_to_{tar}")

    return training_logs_so

### Training

In [ ]:
summary(DANNMBNv3s().to(device), input_size=(channel_size, image_size, image_size))
num_epochs = 50
lr = 1e-3
loss_fn_class = torch.nn.NLLLoss()
loss_fn_domain = torch.nn.NLLLoss()
srcs = ["amazon", "dslr", "webcam"]
tars = ["amazon", "dslr", "webcam"]
for src in srcs:
    for tar in tars:
        if src != tar:
            model = DANNMBNv3s().to(device)
            src_dataloader, tar_dataloader = get_loader_DA(src, tar)
            optimizer = optim.Adam(model.parameters(), lr)
            steps_per_epoch = max(len(src_dataloader), len(tar_dataloader))
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=lr,  # Max learning rate during the cycle
                steps_per_epoch=steps_per_epoch,
                epochs=num_epochs,
                anneal_strategy="cos"  # Can also try 'cos' for cosine annealing
            )
            plot_graphDA(train_model_with_balanced_batches_DANN(model, optimizer, scheduler, loss_fn_class, loss_fn_domain, src, tar, device, num_epochs=num_epochs, batch_size=64), f"DANN {src} -> {tar}")
        else: print("Source and Target are the same. Skiping...")


Streaming output truncated to the last 5000 lines.
[14/33] Loss: 1.3432
[15/33] Loss: 1.4044
[16/33] Loss: 1.4881
[17/33] Loss: 1.3512
[18/33] Loss: 1.2544
[19/33] Loss: 1.4965
[20/33] Loss: 1.3022
[21/33] Loss: 1.2550
[22/33] Loss: 1.3485
[23/33] Loss: 1.3157
[24/33] Loss: 1.3171
[25/33] Loss: 1.3430
[26/33] Loss: 1.3305
[27/33] Loss: 1.3705
[28/33] Loss: 1.2899
[29/33] Loss: 1.5059
[30/33] Loss: 1.3082
[31/33] Loss: 1.2883
[32/33] Loss: 1.3414
[33/33] Loss: 1.4896
Epoch: 48 | Source Accuracy: 0.9985, Target Accuracy: 0.5207, Loss: 1.3696
Epoch 0049 / 0050
[1/33] Loss: 1.3821
[2/33] Loss: 1.4603
[3/33] Loss: 1.3449
[4/33] Loss: 1.3193
[5/33] Loss: 1.3540
[6/33] Loss: 1.3097
[7/33] Loss: 1.2994
[8/33] Loss: 1.3497
[9/33] Loss: 1.2594
[10/33] Loss: 1.4231
[11/33] Loss: 1.5293
[12/33] Loss: 1.3237
[13/33] Loss: 1.2836
[14/33] Loss: 1.3267
[15/33] Loss: 1.3580
[16/33] Loss: 1.5088
[17/33] Loss: 1.3139
[18/33] Loss: 1.3432
[19/33] Loss: 1.2935
[20/33] Loss: 1.3283
[21/33] Loss: 1.3593
[22/

In [ ]:
zip_directory('./cp', 'DANN_Model.zip')
zip_log_files("DANN_LOG.zip")

## Coral

### Training Loop Function

In [ ]:
# Function to compute the CORAL loss
class CorrelationAlignmentLoss(nn.Module):
    def forward(self, source, target):
        d = source.size(1)  # Number of features

        # Compute covariance matrices
        source_cov = (source.t() @ source) / (source.size(0) - 1)
        target_cov = (target.t() @ target) / (target.size(0) - 1)

        # Compute CORAL loss as the Frobenius norm between covariance matrices
        coral_loss = torch.mean((source_cov - target_cov) ** 2) / (4 * d * d)

        return coral_loss

# Balancing dataloaders
def create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size, blur_sigma=2):
    src_size = len(src_dataloader.dataset)
    tar_size = len(tar_dataloader.dataset)

    # Define a blur transformation
    blur_transform = transforms.GaussianBlur(kernel_size=(5, 5), sigma=blur_sigma)

    def blur_images(batch):
        images, labels = batch
        # Apply blur to images
        blurred_images = blur_transform(images)
        return blurred_images, labels

    # Ensure iterators are reset appropriately for smaller dataset
    if src_size > tar_size:
        Dl_source_iter = iter(src_dataloader)
        Dl_target_iter = itertools.cycle(tar_dataloader)  # Cycle the smaller dataset
        max_batches = len(src_dataloader)

        # Apply blur to the cycled target images
        Dl_target_iter = (blur_images(batch) for batch in Dl_target_iter)
    else:
        Dl_source_iter = itertools.cycle(src_dataloader)  # Cycle the smaller dataset
        Dl_target_iter = iter(tar_dataloader)
        max_batches = len(tar_dataloader)

        # Apply blur to the cycled source images
        Dl_source_iter = (blur_images(batch) for batch in Dl_source_iter)

    return Dl_source_iter, Dl_target_iter, max_batches

# CORAL training loop
def train_model_with_balanced_batches_CORAL(model, optimizer, loss_fn_class, src, tar, device, num_epochs=30, batch_size=64, lambda_coral=0.01):
    training_logs_coral = {"train_loss": [],  "train_src_acc": [], "train_tar_acc": []}


    src_dataloader, tar_dataloader = get_loader_DA(src, tar)
    model_save_path = f"SO_{src}_to_{tar}.pth"
    best_vloss = 1_000_000.
    for epoch_idx in range(num_epochs):
        train_loss, train_src_correct, train_tar_correct = 0, 0, 0
        actual_src_samples, actual_tar_samples = 0, 0
        print(f'Epoch {epoch_idx+1:04d} / {num_epochs:04d}', end='\n============\n')

        # Create balanced iterators
        Dl_source_iter, Dl_target_iter, max_batches = create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size)

        for batch_idx in range(max_batches):
            optimizer.zero_grad()

            # Get next batch from source and target
            X_s, y_s = next(Dl_source_iter)
            X_t, y_t = next(Dl_target_iter)

            # Ensure the batches have the same size by taking the minimum batch size
            if X_s.shape[0] != X_t.shape[0]:
                min_bs = min(X_s.shape[0], X_t.shape[0])
                X_s, y_s = X_s[:min_bs], y_s[:min_bs]
                X_t, y_t = X_t[:min_bs], y_t[:min_bs]

            # Track the number of samples processed
            actual_src_samples += y_s.size(0)
            actual_tar_samples += y_t.size(0)

            # Send data to device (GPU or CPU)
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            # Forward pass for classification on the source domain
            class_pred_s = model(X_s)

            # Forward pass for target domain (used for CORAL loss)
            class_pred_t = model(X_t)

            # Compute source classification loss
            loss_src = loss_fn_class(class_pred_s, y_s)

            # Compute CORAL loss between source and target features
            loss_coral = CorrelationAlignmentLoss()(class_pred_s, class_pred_t)

            # Combine losses: classification + CORAL
            loss = loss_src + lambda_coral * loss_coral

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute accuracy for source and target domains
            with torch.no_grad():
                class_prediction_s = model(X_s)
                class_prediction_t = model(X_t)

            train_src_correct += (class_prediction_s.argmax(1) == y_s).float().sum().item()
            train_tar_correct += (class_prediction_t.argmax(1) == y_t).float().sum().item()

            train_loss += loss.item()

            # Print batch info
            print(f'[{batch_idx+1}/{max_batches}] class loss: {loss_src.item():.4f} CORAL_loss: {loss_coral.item():.4f}')

        # Calculate and store metrics for the epoch
        training_logs_coral["train_loss"].append(train_loss / max_batches)
        training_logs_coral["train_src_acc"].append(train_src_correct / actual_src_samples)
        training_logs_coral["train_tar_acc"].append(train_tar_correct / actual_tar_samples)

        print(f'Epoch: {epoch_idx+1} || '
              f'Train_src_acc: {train_src_correct / actual_src_samples:.4f}, '
              f'Train_tar_acc: {train_tar_correct / actual_tar_samples:.4f}, '
              f'Train_loss: {train_loss / max_batches:.4f}')
        if train_loss < best_vloss:
            best_vloss = train_loss
            path_save_cp = './cp/'
            if not os.path.exists(path_save_cp): os.mkdir(path_save_cp)
            torch.save(model.state_dict(), path_save_cp+model_save_path)


    save_training_logs(training_logs_coral, f"LOG_CORAL_{src}_to_{tar}")

    return training_logs_coral

model = models.mobilenet_v3_small(pretrained=True)
print("Original Classifier:", model.classifier)
model.classifier[-1] = nn.Linear(in_features=1024, out_features=31, bias=True)
print("Mod Classifier:", model.classifier)
model.to(device)
summary(model.to('cpu'), input_size=(3, 224, 224), device='cpu')


model.to('cuda')
# shape and tensor test
# x0_s, y0_s = next(iter(Dl_source))
# x0_t, y0_t = next(iter(Dl_target))
#
# print('Source domain input: ', x0_s.shape, y0_s.shape)
# print('Target domain input: ', x0_t.shape, y0_t.shape)
#
# # Test forward pass: get class prediction and domain prediction
# y0_pred_s_class, y0_pred_s_domain = model(x0_s)
# y0_pred_t_class, y0_pred_t_domain = model(x0_t)
#
# print('label prediction from classification: ', y0_pred_t_class, y0_pred_t_class.shape)
# print('domain prediction from classification: ', y0_pred_t_domain, y0_pred_t_domain.shape)

# Optimizer and cost function
optimizer = optim.Adam(model.parameters(), lr)
# loss_fn_class = torch.nn.NLLLoss()
# loss_fn_domain = torch.nn.NLLLoss()
loss_fn_class = torch.nn.NLLLoss()

plot_graph(train_model_with_balanced_batches_CORAL(model, optimizer, loss_fn_class, webcam_tl, amazon_tl, device, num_epochs=30, batch_size=32, lambda_coral=1.0))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Mod Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=31, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 16, 56, 56]             144
       BatchNorm2d-5           [-1, 16, 56, 56]              32
              ReLU-6           [-1, 16, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
    

### Training

In [ ]:
# Optimizer and initial learning rate
lr = 1e-4  # You can start with this low learning rate
optimizer = optim.Adam(model.parameters(), lr=lr)

# Define a scheduler to reduce learning rate when there's no improvement
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

class CorrelationAlignmentLoss(nn.Module):
    def forward(self, source, target):
        d = source.size(1)  # Number of features

        # Compute covariance matrices
        source_cov = (source.t() @ source) / (source.size(0) - 1)
        target_cov = (target.t() @ target) / (target.size(0) - 1)

        # Compute CORAL loss as the Frobenius norm between covariance matrices
        coral_loss = torch.mean((source_cov - target_cov) ** 2) / (4 * d * d)

        return coral_loss


# Function to adjust learning rate for warm-up
def adjust_learning_rate(optimizer, epoch, lr, warmup_epochs=5):
    """Adjusts the learning rate during the warm-up phase."""
    if epoch < warmup_epochs:
        new_lr = lr * (epoch + 1) / warmup_epochs  # Gradually increase LR
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_lr

# CORAL training loop with LR fine-tuning
def train_model_with_balanced_batches_CORAL(model, optimizer, loss_fn_class, src_dataloader, tar_dataloader, device, num_epochs=30, batch_size=32, lambda_coral=0.01, lr=1e-4):
    training_logs_coral = {"train_loss": [],  "train_src_acc": [], "train_tar_acc": []}

    for epoch_idx in range(num_epochs):
        # Adjust the learning rate (warm-up phase for the first few epochs)
        adjust_learning_rate(optimizer, epoch_idx, lr)

        train_loss, train_src_correct, train_tar_correct = 0, 0, 0
        actual_src_samples, actual_tar_samples = 0, 0
        print(f'Epoch {epoch_idx+1:04d} / {num_epochs:04d}', end='\n============\n')

        # Create balanced iterators
        Dl_source_iter, Dl_target_iter, max_batches = create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size)

        for batch_idx in range(max_batches):
            optimizer.zero_grad()

            # Get next batch from source and target
            X_s, y_s = next(Dl_source_iter)
            X_t, y_t = next(Dl_target_iter)

            # Ensure the batches have the same size by taking the minimum batch size
            if X_s.shape[0] != X_t.shape[0]:
                min_bs = min(X_s.shape[0], X_t.shape[0])
                X_s, y_s = X_s[:min_bs], y_s[:min_bs]
                X_t, y_t = X_t[:min_bs], y_t[:min_bs]

            # Track the number of samples processed
            actual_src_samples += y_s.size(0)
            actual_tar_samples += y_t.size(0)

            # Send data to device (GPU or CPU)
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            features_s = model.features(X_s)
            features_t = model.features(X_t)

            # Forward pass for classification on the source domain
            class_pred_s = model(X_s)

            # Forward pass for target domain (used for CORAL loss)
            class_pred_t = model(X_t)

            # Compute source classification loss
            loss_src = loss_fn_class(class_pred_s, y_s)

            # Compute CORAL loss between source and target features
            loss_coral = CorrelationAlignmentLoss()(features_s, features_t)

            # Combine losses: classification + CORAL
            loss = loss_src + lambda_coral * loss_coral

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute accuracy for source and target domains
            with torch.no_grad():
                class_prediction_s = model(X_s)
                class_prediction_t = model(X_t)

            train_src_correct += (class_prediction_s.argmax(1) == y_s).float().sum().item()
            train_tar_correct += (class_prediction_t.argmax(1) == y_t).float().sum().item()

            train_loss += loss.item()

            # Print batch info
            print(f'[{batch_idx+1}/{max_batches}] class loss: {loss_src.item():.4f} CORAL_loss: {loss_coral.item():.4f}')

        # Calculate and store metrics for the epoch
        training_logs_coral["train_loss"].append(train_loss / max_batches)
        training_logs_coral["train_src_acc"].append(train_src_correct / actual_src_samples)
        training_logs_coral["train_tar_acc"].append(train_tar_correct / actual_tar_samples)

        print(f'Epoch: {epoch_idx+1} || '
              f'Train_src_acc: {train_src_correct / actual_src_samples:.4f}, '
              f'Train_tar_acc: {train_tar_correct / actual_tar_samples:.4f}, '
              f'Train_loss: {train_loss / max_batches:.4f}')

        # Step the learning rate scheduler based on the current loss
        scheduler.step(train_loss)

    return training_logs_coral

# Fine-tuned learning rate
lr = 1e-4  # Lower base learning rate for fine-tuning
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[-1] = nn.Linear(in_features=1024, out_features=31, bias=True)
model.to(device)

# Run training with fine-tuned learning rate
train_model_with_balanced_batches_CORAL(model, optimizer, loss_fn_class, webcam_tl, amazon_tl, device, num_epochs=30, batch_size=32, lambda_coral=1.0)


NameError: name 'model' is not defined

## Baseline Training

### Training Baseline

In [ ]:

def train_baseline_model(model, optimizer, scheduler, loss_fn, sd,
                         device, num_epochs=50, batch_size=64):
    """
    Train the baseline model and log training/validation metrics.

    Args:
        model: The model to train.
        optimizer: Optimizer for the model.
        loss_fn: Loss function (e.g., CrossEntropyLoss).
        training_loader: DataLoader for the training set.
        validation_loader: DataLoader for the validation set.
        device: Device to run the training on (e.g., 'cuda' or 'cpu').
        num_epochs: Number of training epochs (default: 10).
        batch_size: Batch size used in training (default: 64).
        lr: Learning rate for the optimizer (default: 0.001).

    Returns:
        training_logs_student: A dictionary with training/validation loss and accuracy logs.
    """

    training_loader, validation_loader = get_loader(sd)
    t_0 = time.time()
    best_vloss = 1_000_000.  # Initialize best validation loss to a large value
    training_logs_student = {"train_student_loss": [], "val_student_loss": [], "train_student_acc": [], "val_student_acc": []}
    model_name = f"Baseline_{sd}.pth"

    for epoch in range(num_epochs):
        train_loss, train_correct = 0, 0
        model.train()  # Set model to training mode
        print(f'Epoch {epoch+1:04d} / {num_epochs:04d}', end='\n============\n')


        for i, data in enumerate(training_loader):
            optimizer.zero_grad()  # Zero the gradients

            X, y = data[0].to(device), data[1].to(device)  # Send data to the device
            yhat = model(X)  # Forward pass
            loss = loss_fn(yhat, y)  # Calculate loss

            loss.backward()  # Backward pass
            optimizer.step()  # Optimize model parameters
            scheduler.step()

            with torch.no_grad():
                class_prediction = model(X)  # Predictions for accuracy calculation

            print(f'[{i+1}/{len(training_loader)}] student class loss: {loss.item():.4f}')

            train_loss += loss.item()
            train_correct += (class_prediction.argmax(1) == y).float().sum().item()

        # Log training metrics
        training_logs_student["train_student_loss"].append(train_loss / len(training_loader))
        training_logs_student["train_student_acc"].append(train_correct / len(training_loader.dataset))

        # Validation step
        valid_loss, valid_correct = 0, 0
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            for i, vdata in enumerate(validation_loader):
                vinputs, vlabels = vdata[0].to(device), vdata[1].to(device)
                voutputs = model(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                valid_loss += vloss.item()
                valid_correct += (voutputs.argmax(1) == vlabels).float().sum().item()

        # Log validation metrics
        training_logs_student["val_student_loss"].append(valid_loss / len(validation_loader))
        training_logs_student["val_student_acc"].append(valid_correct / len(validation_loader.dataset))

        # Print epoch summary
        print(f'Epochs {epoch+1}'.ljust(10),
              f'train loss {training_logs_student["train_student_loss"][-1]:.5f}',
              f'train acc {training_logs_student["train_student_acc"][-1]:.5f}',
              f'validate loss {training_logs_student["val_student_loss"][-1]:.5f}',
              f'validate acc {training_logs_student["val_student_acc"][-1]:.5f}',
              )
        print("-" * 80)

        # Save the best model based on validation loss
        if valid_loss < best_vloss:
            best_vloss = valid_loss
            path_save_cp = './cp/'
            if not os.path.exists(path_save_cp):
                os.mkdir(path_save_cp)
            torch.save(model.state_dict(), os.path.join(path_save_cp, model_name))

    t_end = time.time() - t_0
    print(f"Time consumption for student net (device:{device}): {t_end} sec")

    # Save the training logs to a JSON file
    save_training_logs(training_logs_student, f"LOG_BL_{sd}")


    return training_logs_student

In [ ]:
model = models.mobilenet_v3_small(pretrained=True).to(device)
summary(model, input_size=(channel_size, image_size, image_size))
num_epochs = 50
lr = 1e-3
loss_fn = torch.nn.CrossEntropyLoss()
sds = ["amazon", "dslr", "webcam"]
for sd in sds:
    model = models.mobilenet_v3_small(pretrained=True).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    tl, _ =get_loader(sd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        steps_per_epoch=len(tl),
        epochs=num_epochs,
        anneal_strategy="cos",
    )
    plot_graph(train_baseline_model(model, optimizer, scheduler, loss_fn, sd, device, num_epochs=num_epochs, batch_size=64), f"Baseline {sd}")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 16, 56, 56]             144
       BatchNorm2d-5           [-1, 16, 56, 56]              32
              ReLU-6           [-1, 16, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             136
              ReLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 16, 1, 1]             144
      Hardsigmoid-11             [-1, 16, 1, 1]               0
SqueezeExcitation-12           [-1, 16, 56, 56]               0
           Conv2d-13           [-1, 16, 56, 56]             256
      BatchNorm2d-14           [-1, 16,

## Pruning

### Funtion

In [ ]:
def calculate_layer_sparsity(model):
    total_zeros = 0
    total_elements = 0
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            # Check if the module has a weight attribute
            if hasattr(module, 'weight'):
                weight = module.weight
                zeros = torch.sum(weight == 0)
                total_zeros += zeros
                total_elements += weight.nelement()
                sparsity = 100.0 * float(zeros) / float(weight.nelement())
                print(f"Sparsity in {name}.weight: {sparsity:.2f}%")

    # Global sparsity calculation
    global_sparsity = 100.0 * float(total_zeros) / float(total_elements)
    print(f"Global sparsity: {global_sparsity:.2f}%")

def save_pruned_model(model, path):
    model_state = model.state_dict()
    pruning_params = {}
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            if prune.is_pruned(module):  # Check if the module has been pruned
                pruning_params[name] = {
                    'weight_mask': module.weight_mask,
                    'original_weight': module.weight_orig
                }

    # Save both the model state and pruning parameters
    torch.save({'model_state_dict': model_state, 'pruning_params': pruning_params}, path)

def prune_evaluate_save(model, model_path, train_loader, test_loader, save_path):
    # Load the pre-trained weights
    PATH_CP = './cp/'
    model.load_state_dict(torch.load(PATH_CP + model_path))
    model.to(device)
    _, _, _, acc = inference(model, test_loader)

    # Display model summary
    summary(model, input_size=(channel_size, image_size, image_size))

    # Count initial FLOPs and parameters
    init_flop = count_model_param_flops(model=model.eval(), input_res=224, multiply_adds=True)  / 1e6

    # Apply structured pruning
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=0.2, n=2, dim=0)

    # Remove pruning reparameterization
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.remove(module, 'weight')

    # Calculate sparsity
    calculate_layer_sparsity(model)

    # Count FLOPs and parameters after pruning
    post_flop = count_model_param_flops(model=model.eval(), input_res=224, multiply_adds=True)  / 1e6

    # Save pruned model
    save_pruned_model(model, save_path)
    _, _, _, acc = inference(model, test_loader)
    return init_flop, post_flop, acc




def load_pruned_model(model_class, path, device):
    # Load the checkpoint
    checkpoint = torch.load(path)

    # Initialize the model
    model = model_class().to(device)

    # Load the state_dict
    model.load_state_dict(checkpoint['model_state_dict'])

    # Reapply the pruning mask
    pruning_params = checkpoint['pruning_params']
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            if name in pruning_params:
                prune.custom_from_mask(module, name='weight', mask=pruning_params[name]['weight_mask'])
                # Replace the pruned weight with the original one (reparameterize)
                module.weight_orig = pruning_params[name]['original_weight']

    return model

# Example usage
# student_fn_loaded = load_pruned_model(DANNMBNv3s(), 'pruned_model.pth', device)

### Pruning Loop

In [ ]:
lst = [
    [DANNMBNv3s(), "DANN(Teacher)_amazon_to_dslr.pth", amazon_tl, dslr_tl, True],
    [DANNMBNv3s(), "DANN(Teacher)_amazon_to_webcam.pth", amazon_tl, webcam_tl, True],
    [DANNMBNv3s(), "DANN(Teacher)_dslr_to_amazon.pth", dslr_tl, amazon_tl, True],
    [DANNMBNv3s(), "DANN(Teacher)_dslr_to_webcam.pth", dslr_tl, webcam_tl, True],
    [DANNMBNv3s(), "DANN(Teacher)_webcam_to_amazon.pth", webcam_tl, amazon_tl, True],
    [DANNMBNv3s(), "DANN(Teacher)_webcam_to_dslr.pth", webcam_tl, dslr_tl, True],
]

In [ ]:
# Loop through your list and apply the function
csv_output = "pruning_result.csv"
PATH_CP = './cp/'
headers = ['model_name', 'pre-pruning', 'post-pruning', 'acc']
with open(csv_output, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()  # Write header
    for idx, (model, model_path, train_loader, test_loader, to_save) in enumerate(lst):
        results = {}
        save_path = f"pruned_model_{idx}.pth"
        results["model_name"] = model_path.rstrip(".pth")
        results["pre-pruning"], results["post-pruning"], results["acc"] = prune_evaluate_save(model, model_path, train_loader, test_loader, save_path)


            # Write the results to CSV
        writer.writerow(results)

    print(f"Results saved to {csv_output}")


<ipython-input-49-b57f027c97d5>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH_CP + model_path))


[(Inference || test loss: 4.557110786437988] [accuracy_test: 53.080000000000005 %]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 16, 56, 56]             144
       BatchNorm2d-5           [-1, 16, 56, 56]              32
              ReLU-6           [-1, 16, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             136
              ReLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 16, 1, 1]             144
      Hardsigmoid-11             [-1, 16, 1, 1]               0
SqueezeExcitation-12           [-1, 16, 56, 56]               0
           Conv2d-13

In [ ]:
def create_balanced_dataloaders(src_dataloader, tar_dataloader, batch_size, blur_sigma=2):
    src_size = len(src_dataloader.dataset)
    tar_size = len(tar_dataloader.dataset)

    # Define a blur transformation
    blur_transform = transforms.GaussianBlur(kernel_size=(5, 5), sigma=blur_sigma)

    def blur_images(batch):
        images, labels = batch
        # Apply blur to images
        blurred_images = blur_transform(images)
        return blurred_images, labels

    # Ensure iterators are reset appropriately for smaller dataset
    if src_size > tar_size:
        Dl_source_iter = iter(src_dataloader)
        Dl_target_iter = itertools.cycle(tar_dataloader)  # Cycle the smaller dataset
        max_batches = len(src_dataloader)

        # Apply blur to the cycled target images
        Dl_target_iter = (blur_images(batch) for batch in Dl_target_iter)
    else:
        Dl_source_iter = itertools.cycle(src_dataloader)  # Cycle the smaller dataset
        Dl_target_iter = iter(tar_dataloader)
        max_batches = len(tar_dataloader)

        # Apply blur to the cycled source images
        Dl_source_iter = (blur_images(batch) for batch in Dl_source_iter)

    return Dl_source_iter, Dl_target_iter, max_batches



def train_model_with_balanced_batches_DANN_Pruned(model, optimizer, scheduler, loss_fn_class, loss_fn_domain,
                                           src, tar, device,
                                           num_epochs=30, batch_size=64, name="Test"):
    """
    Train the DANN model using balanced batches from both source and target domain dataloaders.
    Args:
        model: The DANN model (including feature extractor and domain classifier).
        optimizer: Optimizer for the model.
        loss_fn_class: Loss function for classification (e.g., cross-entropy).
        loss_fn_domain: Loss function for domain classification (e.g., binary cross-entropy).
        src_dataloader: DataLoader for the source domain.
        tar_dataloader: DataLoader for the target domain.
        device: Device to run the training on (e.g., 'cuda' or 'cpu').
        num_epochs: Number of training epochs.
        batch_size: Batch size used in training.

    Returns:
        training_logs_so: A dictionary with training loss and accuracies for source and target domains.
    """

    # Logs to track training performance
    training_logs_so = {"train_loss": [], "train_src_acc": [], "train_tar_acc": []}
    model_save_path = name
    best_vloss = 1_000_000.

    for epoch_idx in range(num_epochs):
        train_loss, train_src_correct, train_tar_correct = 0, 0, 0
        actual_src_samples, actual_tar_samples = 0, 0  # Track the actual number of processed samples

        print(f'Epoch {epoch_idx+1:04d} / {num_epochs:04d}', end='\n============\n')

        # Create balanced iterators for source and target domain
        Dl_source_iter, Dl_target_iter, max_batches = create_balanced_dataloaders(src, tar, batch_size)

        for batch_idx in range(max_batches):
            optimizer.zero_grad()

            # Get next batch from source and target datasets
            X_s, y_s = next(Dl_source_iter)
            X_t, y_t = next(Dl_target_iter)

            # Ensure the batches have the same size by taking the minimum batch size
            if X_s.shape[0] != X_t.shape[0]:
                min_bs = min(X_s.shape[0], X_t.shape[0])
                X_s, y_s = X_s[:min_bs], y_s[:min_bs]
                X_t, y_t = X_t[:min_bs], y_t[:min_bs]

            # Track the number of samples processed
            actual_src_samples += y_s.size(0)
            actual_tar_samples += y_t.size(0)

            # Send data to device (GPU or CPU)
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            # Dynamic adjustment of grl_lambda
            p = float(batch_idx + epoch_idx * max_batches) / (num_epochs * max_batches)
            grl_lambda = (2 / (1 + np.exp(-10 * p)) - 1) * 0.5

            # Prepare labels for domain classification (source domain = 0, target domain = 1)
            y_s_domain = torch.zeros(X_s.size(0), dtype=torch.long).to(device)
            y_t_domain = torch.ones(X_t.size(0), dtype=torch.long).to(device)

            # Forward pass through the model
            class_prediction_s, domain_prediction_s = model(X_s, grl_lambda)
            _, domain_prediction_t = model(X_t, grl_lambda)

            # Calculate classification and domain losses
            loss_s_label = loss_fn_class(class_prediction_s, y_s)  # Source classification loss
            loss_s_domain = loss_fn_domain(domain_prediction_s, y_s_domain)  # Source domain loss
            loss_t_domain = loss_fn_domain(domain_prediction_t, y_t_domain)  # Target domain loss

            # Combine the losses with a weight for domain loss
            loss = loss_s_label + loss_s_domain + loss_t_domain

            # Backpropagation
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Accumulate the training loss
            train_loss += loss.item()

            # Compute accuracy for source domain classification
            train_src_correct += (class_prediction_s.argmax(1) == y_s).float().sum().item()

            # Compute accuracy for target domain classification (even though target labels are not used in training)
            with torch.no_grad():
                class_prediction_t, _ = model(X_t, grl_lambda)
            train_tar_correct += (class_prediction_t.argmax(1) == y_t).float().sum().item()

            # Print batch info
            print(f'[{batch_idx+1}/{max_batches}] Loss: {loss.item():.4f}')

        # Calculate and store metrics for the epoch
        training_logs_so["train_loss"].append(train_loss / max_batches)
        training_logs_so["train_src_acc"].append(train_src_correct / actual_src_samples)
        training_logs_so["train_tar_acc"].append(train_tar_correct / actual_tar_samples)

        print(f'Epoch: {epoch_idx+1} | '
              f'Source Accuracy: {train_src_correct / actual_src_samples:.4f}, '
              f'Target Accuracy: {train_tar_correct / actual_tar_samples:.4f}, '
              f'Loss: {train_loss / max_batches:.4f}')
        if train_loss < best_vloss:
            best_vloss = train_loss
            path_save_cp = './cp/'
            if not os.path.exists(path_save_cp): os.mkdir(path_save_cp)
            torch.save(model.state_dict(), path_save_cp+model_save_path)


    save_training_logs(training_logs_so, f"LOG_DANN_Pruned_{name.rstrip('.pth')}")

    return training_logs_so

In [ ]:
lr = 1e-3
num_epochs = 10

for i in range(6):
    model = load_pruned_model(DANNMBNv3s, f"pruned_model_{i}.pth", device)
    src_dataloader, tar_dataloader = lst[i][2], lst[i][3]
    optimizer = optim.Adam(model.parameters(), lr)
    steps_per_epoch = max(len(src_dataloader), len(tar_dataloader))
    loss_fn_class = torch.nn.NLLLoss()
    loss_fn_domain = torch.nn.NLLLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,  # Max learning rate during the cycle
        steps_per_epoch=steps_per_epoch,
        epochs=num_epochs,
        anneal_strategy="cos"  # Can also try 'cos' for cosine annealing
    )
    plot_graphDA(train_model_with_balanced_batches_DANN_Pruned(model, optimizer, scheduler, loss_fn_class, loss_fn_domain, src_dataloader, tar_dataloader, device, num_epochs=num_epochs, batch_size=64, name=f"pruned_model_{i}.pth"), f"DANN_Pruned {i}")
    inference(model, tar_dataloader)


<ipython-input-49-b57f027c97d5>:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Epoch 0001 / 0010
[1/33] Loss: 5.4421
[2/33] Loss: 5.1149
[3/33] Loss: 4.7143
[4/33] Loss: 4.2963
[5/33] Loss: 4.8800
[6/33] Loss: 3.7760
[7/33] Loss: 4.1838
[8/33] Loss: 4.2391
[9/33] Loss: 3.5363
[10/33] Loss: 3.7206
[11/33] Loss: 3.6576
[12/33] Loss: 3.3344
[13/33] Loss: 2.9847
[14/33] Loss: 3.5251
[15/33] Loss: 3.7235
[16/33] Loss: 2.9432
[17/33] Loss: 3.0608
[18/33] Loss: 2.8301
[19/33] Loss: 2.7064
[20/33] Loss: 3.2555
[21/33] Loss: 2.5630
[22/33] Loss: 2.9048
[23/33] Loss: 2.8706
[24/33] Loss: 2.5785
[25/33] Loss: 2.6430
[26/33] Loss: 2.8405
[27/33] Loss: 3.0477
[28/33] Loss: 2.9205
[29/33] Loss: 2.5239
[30/33] Loss: 2.5941
[31/33] Loss: 2.2750
[32/33] Loss: 2.2542
[33/33] Loss: 2.5836
Epoch: 1 | Source Accuracy: 0.4861, Target Accuracy: 0.2800, Loss: 3.3492
Epoch 0002 / 0010
[1/33] Loss: 1.9380
[2/33] Loss: 2.4725
[3/33] Loss: 1.9991
[4/33] Loss: 2.0660
[5/33] Loss: 1.6108
[6/33] Loss: 1.6710
[7/33] Loss: 2.1205
[8/33] Loss: 1.8324
[9/33] Loss: 1.9721
[10/33] Loss: 1.9970
[11/3

# ONNX

In [ ]:
# Save the ONNX model in a file

# Define model and export the Torch model to onnx format
!pip install onnx
!pip install onnxscript
import onnx
from torch import func
torch_model = load_pruned_model(DANNMBNv3s, f"pruned_model_0.pth", device).to('cpu')
torch_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(
    torch_model,                       # The model to export
    torch_input,                 # A sample input tensor for the export process
    "my_image_classifier.onnx",    # The output file name
    opset_version=18,            # Specify the opset version
    input_names=['input'],       # Names for the input nodes
    output_names=['output']      # Names for the output nodes
)

# Import library and load from ONNX side

onnx_program = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_program)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.5/670.5 kB 33.7 MB/s eta 0:00:00


<ipython-input-49-b57f027c97d5>:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)
<ipython-input-16-028dd41c05bf>:48: TracerWarning: Converting

In [ ]:
# Execute the ONNX model with ONNX Runtime
# !pip install onnxruntime
!pip install onnxruntime-gpu
import onnxruntime

# Create an ONNXRuntime session
ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

# Function to convert PyTorch tensor to NumPy array
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Prepare the inputs for ONNXRuntime (directly convert PyTorch tensor to NumPy)
onnxruntime_input = {k.name: to_numpy(torch_input) for k in ort_session.get_inputs()}

# Run inference with ONNXRuntime
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

# Compare the PyTorch results with the ones from ONNXRuntime
import torch
import numpy as np

# Set the PyTorch model to evaluation mode
torch_model.eval()

# Run inference with PyTorch and ensure no gradients are computed
with torch.no_grad():
    torch_outputs = torch_model(torch_input)

# Squeeze the ONNXRuntime output to remove the batch dimension ([1, 1000] -> [1000])
onnxruntime_output_squeezed = [np.squeeze(output) for output in onnxruntime_outputs]

# Ensure that ONNX output has the correct shape by adding the batch dimension if necessary
for i, onnx_output in enumerate(onnxruntime_output_squeezed):
    if len(onnx_output.shape) == 1:  # If ONNX output is missing the batch dimension
        onnxruntime_output_squeezed[i] = np.expand_dims(onnx_output, axis=0)  # Add batch dimension

# Compare the number of outputs
assert len(torch_outputs) == len(onnxruntime_output_squeezed), "The number of outputs does not match."

# Compare outputs from PyTorch and ONNXRuntime
for torch_output, onnx_output in zip(torch_outputs, onnxruntime_output_squeezed):
    torch.testing.assert_close(torch_output, torch.tensor(onnx_output), rtol=1e-03, atol=1e-05)

print("\n\nPyTorch and ONNX Runtime output matched!")

# Display information about the outputs
print(f"PyTorch Output length: {len(torch_outputs)}")
print(f"PyTorch Sample Output: {torch_outputs}")
print(f"ONNXRuntime Output length: {len(onnxruntime_output_squeezed)}")
print(f"ONNXRuntime Sample output: {onnxruntime_output_squeezed}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00


PyTorch and ONNX Runtime output matched!
PyTorch Output length: 2
PyTorch Sample Output: (tensor([[-2.9416e+01, -1.8039e+01, -1.2913e+01, -3.1296e+01, -4.2207e+01,
         -2.7517e+01, -2.9192e+01, -4.3720e+01, -3.0120e+01, -3.8755e+01,
         -2.6278e+01, -2.9082e+01, -3.4297e+01, -3.2459e+01, -4.0089e+01,
         -2.2068e+01, -3.6917e+01, -2.4688e+01, -3.9485e+01, -2.6960e+01,
         -3.9616e+01, -3.5855e+01, -2.3308e+01, -2.6226e-06, -3.0143e+01,
         -1.6609e+01, -1.6227e+01, -2.0582e+01, -3.5823e+01, -2.8470e+01,
         -3.5352e+01]]), tensor([[-0.7796, -0.6136]]))
ONNXRuntime Output length: 2
ONNXRuntime Sample output: [array([[-2.9415836e+01, -1.8039225e+01, -1.2913147e+01, -3.1295645e+01,
        -4.2206989e+01, -2.7516792e+01, -2.9

In [ ]:
input_shape = (1, 3, 224, 224)
input_data_for_inference = torch.randn(input_shape, dtype=torch.float32).cuda()
input_data_for_inference_fp16 = input_data_for_inference.to(dtype=torch.float16)

In [ ]:
# @title Non-Optimized FP32 (Full Tensor)
import time

input_shape = (1, 3, 224, 224)
output_shape = (1, 10)

#>>>> batch runs session
print("Non-optimizing FP32 bench testing...")
#---Non-optimized---
nonopt_model = load_pruned_model(DANNMBNv3s, f"pruned_model_0.pth", device).cuda().eval()
num_iterations = 10000
total_time = 0.0
with torch.no_grad():
    for i in range(num_iterations):
        start_time = time.time()
        input_data = torch.randn(input_shape).cuda()
        output_data = nonopt_model(input_data)
        end_time = time.time()
        total_time += end_time - start_time
pytorch_fps = num_iterations / total_time
print(f"PyTorch FPS: {pytorch_fps:.2f}")



#---------Inference with FP32--------------
import onnxruntime as ort
# Load the ONNX model and run inference
# session = ort.InferenceSession('my_image_classifier_fp16.onnx', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
# onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
# onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

output_data = nonopt_model(input_data_for_inference)

print(f"\n\nONNXRuntime [FP32] Output length: {len(output_data)}")
print(f"ONNXRuntime [FP32] Sample output: {output_data}")


In [ ]:
# @title Optimized FP16 (Half Tensor)

import os
os.environ["ALLOW_RELEASED_ONNX_OPSET_ONLY"] = "0"
import onnxruntime.backend as backend
print("Optimized model FP16 bench testing...")

# Try using CUDAExecutionProvider and check if it's available
# ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

torch_model_fp16 = torch_model.half()
dummy_input_fp16 = torch.randn(input_shape, dtype=torch.float16)
input_names = ['input']
output_names = ['output']
torch.onnx.export(torch_model_fp16, dummy_input_fp16, './my_image_classifier_fp16.onnx', verbose=False, input_names=input_names, output_names=output_names)

#onnx_model_path_fp16 = f'outputs/{opt["compressed_directory"]}/compressed_student_net_fp16.onnx'
#session_fp16 = ort.InferenceSession(onnx_model_path_fp16, providers=providers)

# Create a engine from the ONNX model and measure inference speed
model_onnx_fp16 = onnx.load('./my_image_classifier_fp16.onnx')
onnx_engine_fp16 = backend.prepare(model_onnx_fp16, device='GPU', provider='CUDAExecutionProvider', float16=True)

num_iterations = 10000
total_time_fp16 = 0.0
print("start inferencing...FP16")
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape, dtype=torch.float16).cuda()
        start_time = time.time()
        output_data = onnx_engine_fp16.run(input_data.cpu().numpy())[0]
        end_time = time.time()
        total_time_fp16 += end_time - start_time
tensor_fps_fp16 = num_iterations /total_time_fp16
#tensor_fps = num_iterations / total_time
print(f"Tensor FPS [FP16]: {tensor_fps_fp16:.2f}")
print(f"Speedup: {tensor_fps_fp16/pytorch_fps:.2f}x")





#---------Inference with FP16--------------
import onnxruntime as ort
# Load the ONNX model and run inference
# session = ort.InferenceSession('my_image_classifier_fp16.onnx', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
# onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
# onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
# input_data = torch.randn(input_shape, dtype=torch.float16).cuda()
output_data = onnx_engine_fp16.run(input_data_for_inference_fp16.cpu().numpy())[0]

print(f"\n\nONNXRuntime [FP16] Output length: {len(output_data)}")
print(f"ONNXRuntime [FP16] Sample output: {output_data}")

Optimized model FP16 bench testing...


<ipython-input-16-028dd41c05bf>:48: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.shape[1] == 1:
<ipython-input-16-028dd41c05bf>:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif x.shape[1] != 3:


*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
start inferencing...FP16
Tensor FPS [FP16]: 322.75
Speedup: 2.39x


ONNXRuntime [FP16] Output length: 1
ONNXRuntime [FP16] Sample output: [[-1.9375e+01 -1.2984e+01 -1.2555e+01 -2.0406e+01 -2.9625e+01 -2.3125e+01
  -2.0766e+01 -3.0000e+01 -1.6609e+01 -2.5797e+01 -1.9875e+01 -1.9719e+01
  -2.2219e+01 -2.0000e+01 -2.8500e+01 -1.3625e+01 -2.3688e+01 -1.9734e+01
  -2.7891e+01 -2.2828e+01 -2.5922e

In [ ]:
def measure_inference_speed(model, input_shape, dtype=torch.float16, num_iterations=10000, device='cuda'):
    total_time = 0.0
    model.eval()  # Set the model to evaluation mode
    input_data = torch.randn(input_shape, dtype=dtype).to(device)

    print(f"Start inferencing with {dtype} precision...")

    # Measure inference speed for the given precision
    with torch.no_grad():
        for i in range(num_iterations):
            start_time = time.time()
            _ = model(input_data)
            end_time = time.time()
            total_time += end_time - start_time

    # Calculate frames per second (FPS)
    tensor_fps = num_iterations / total_time
    print(f"Tensor FPS [{dtype}]: {tensor_fps:.2f}")

    return tensor_fps

In [ ]:
import torch
import torch.quantization as quant
import torch.nn as nn

# Load your pruned model and wrap it for quantization
torch_model = load_pruned_model(DANNMBNv3s, f"pruned_model_0.pth", device).cpu()

# Assuming you installed the required package torchao:
!pip install torchao

from torchao.quantization.quant_api import (
    quantize_,
    int8_weight_only
)

# Apply INT8 weight quantization
quantize_(torch_model, int8_weight_only())

torch_model_int8 = torch_model

# Create input data for inference
input_shape = (1, 3, 224, 224)  # Example input shape
input_data = torch.randn(input_shape)

# Run inference on the quantized model
with torch.no_grad():
    output_data = torch_model_int8(input_data)

print(f"INT8 output: {output_data}")

<ipython-input-49-b57f027c97d5>:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


INT8 output: (tensor([[-2.7958, -2.6695, -3.2461, -3.8750, -3.8860, -3.9304, -3.7531, -5.1361,
         -2.1449, -5.0628, -4.1636, -3.8265, -3.3535, -4.7222, -3.7332, -2.7727,
         -4.4208, -3.9320, -3.7017, -5.6523, -2.6095, -2.8189, -2.8499, -3.6414,
         -3.9278, -3.3038, -4.2750, -3.7516, -3.8467, -3.1405, -4.2758]]), tensor([[-0.9763, -0.4727]]))


In [ ]:
import torch.onnx

# Export the INT8 quantized model to ONNX
dummy_input_int8 = torch.randn(input_shape)
torch.onnx.export(
    torch_model_int8,
    dummy_input_int8,
    './my_image_classifier_int8.onnx',
    verbose=False,
    input_names=['input'],
    output_names=['output']
)

<ipython-input-16-028dd41c05bf>:48: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.shape[1] == 1:
<ipython-input-16-028dd41c05bf>:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif x.shape[1] != 3:


In [ ]:
import time
import onnx
import onnxruntime as ort
from onnxruntime import backend

# Load the ONNX model
model_onnx_int8 = onnx.load('./my_image_classifier_int8.onnx')

# Prepare the ONNX runtime engine for INT8 (using CPU here, as CUDA INT8 support is limited)
onnx_engine_int8 = ort.InferenceSession('./my_image_classifier_int8.onnx', providers=['CPUExecutionProvider'])

# Measure inference speed for INT8
num_iterations = 10000
total_time_int8 = 0.0
print("start inferencing...INT8")
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape).cpu()  # Ensure input is on CPU for INT8
        start_time = time.time()
        output_data = onnx_engine_int8.run(None, {'input': input_data.numpy()})[0]
        end_time = time.time()
        total_time_int8 += end_time - start_time

# Calculate frames per second for INT8
tensor_fps_int8 = num_iterations / total_time_int8
print(f"Tensor FPS [INT8]: {tensor_fps_int8:.2f}")

# Assuming you have tensor_fps_fp16 from the FP16 test
# Calculate the speedup from INT8 to FP16
print(f"Speedup: {tensor_fps_int8 / pytorch_fps:.2f}x")

start inferencing...INT8
Tensor FPS [INT8]: 319.07
Speedup: 2.36x


# Model Grading

In [ ]:
# Define the grade function
def grade(model, model_name, src_val_tl, tar_val_tl, tSNE=False, adist=False, conf=True, score=True, nmi_ri=True):
    PATH = PATH_CP + model_name
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the model
    model = model.to(device)
    model.load_state_dict(torch.load(PATH))

    stored_lbs, stored_preds, avg_loss, accuracy = inference(model, tar_val_tl)
    source_feature, s_labels = collect_feature(src_val_tl, model, device)
    target_feature, t_labels = collect_feature(tar_val_tl, model, device)

    # Prepare directory to save outputs
    output_dir = './output'
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Results dictionary to store metrics for CSV output
    results = {
        'model_name': model_name,
        'accuracy': accuracy,
        'loss': avg_loss
    }

    if tSNE:
        model.to('cpu')
        # plot t-SNE
        tSNE_filename = os.path.join(output_dir, f'tSNE_{model_name.rstrip(".pth")}.png')
        visualize_class_n_domain(source_feature, target_feature, s_labels, t_labels, tSNE_filename)
        print(f"Saving t-SNE to {tSNE_filename}")

    if adist:
        A_distance = Adist_calculate(source_feature, target_feature, 'cpu')
        print(f"A-distance = {A_distance}")
        results['A_distance'] = A_distance

    lb = stored_lbs.cpu()
    prd = stored_preds.cpu()

    if conf:
        class_names = [f'class_{i}' for i in range(0, 31)]
        cm_target = confusion_matrix(y_true=lb, y_pred=prd, labels=np.arange(num_classes), normalize='true')

        # Save the confusion matrix
        plt.figure()
        plot_confusion_matrix(cm_target, classes=class_names, normalize=True,
                              title=f'Confusion Matrix for {model_name.rstrip(".pth")}',
                              cmap=plt.cm.Blues, show_labels=False, show_numbers=False)
        cm_filename = os.path.join(output_dir, f'CM_{model_name.rstrip(".pth")}.png')
        plt.savefig(cm_filename)
        print(f"Confusion Matrix saved to {cm_filename}")

    if score:
        precision, recall, fscore, _ = precision_recall_fscore_support(lb, prd, average='weighted', zero_division=0)
        print(f"Precision: {precision}, Recall: {recall}, F-score: {fscore}")
        results['precision'] = precision
        results['recall'] = recall
        results['fscore'] = fscore

    if nmi_ri:
        nmi_score = normalized_mutual_info_score(labels_true=lb, labels_pred=prd, average_method='arithmetic')
        ri_score = adjusted_rand_score(labels_true=lb, labels_pred=prd)
        print(f"NMI score: {nmi_score}, RI score: {ri_score}")
        results['nmi_score'] = nmi_score
        results['ri_score'] = ri_score

    return results

In [ ]:

# Function to evaluate all models in the loop
def evaluate_models(lst, csv_output='evaluation_results.csv'):
    # List of headers for the CSV file
    headers = ['model_name', 'accuracy', 'loss', 'A_distance', 'precision', 'recall', 'fscore', 'nmi_score', 'ri_score']

    # Open the CSV file for writing results
    with open(csv_output, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()  # Write header

        for model, model_name, src_val_tl, tar_val_tl, tSNE, adist in lst:
            # Grade the model and get the results
            results = grade(model, model_name, src_val_tl, tar_val_tl, tSNE=tSNE, adist=adist)

            # Write the results to CSV
            writer.writerow(results)

    print(f"Results saved to {csv_output}")

# Example list of models and settings (as per your code)
PATH_CP = './cp/'
lst = [
    [get_model_SO(), "SO_amazon_to_dslr.pth", amazon_vl, dslr_tl, True, True],
    [get_model_SO(), "SO_amazon_to_webcam.pth", amazon_vl, webcam_vl, False, True],
    [get_model_SO(), "SO_dslr_to_amazon.pth", dslr_vl, amazon_vl, False, True],
    [get_model_SO(), "SO_dslr_to_webcam.pth", dslr_vl, webcam_vl, True, True],
    [get_model_SO(), "SO_webcam_to_amazon.pth", webcam_vl, amazon_vl, True, True],
    [get_model_SO(), "SO_webcam_to_dslr.pth", webcam_vl, dslr_vl, False, True],

    [DANNMBNv3s(), "DANN(Teacher)_amazon_to_dslr.pth", amazon_vl, dslr_vl, True, True],
    [DANNMBNv3s(), "DANN(Teacher)_amazon_to_webcam.pth", amazon_vl, webcam_vl, False, True],
    [DANNMBNv3s(), "DANN(Teacher)_dslr_to_amazon.pth", dslr_vl, amazon_vl, False, True],
    [DANNMBNv3s(), "DANN(Teacher)_dslr_to_webcam.pth", dslr_vl, webcam_vl, True, True],
    [DANNMBNv3s(), "DANN(Teacher)_webcam_to_amazon.pth", webcam_vl, amazon_vl, True, True],
    [DANNMBNv3s(), "DANN(Teacher)_webcam_to_dslr.pth", webcam_vl, dslr_vl, False, True],

    # [models.mobilenet_v3_small(pretrained=True), "Baseline_amazon.pth", amazon_vl, amazon_vl, False, False],
    # [models.mobilenet_v3_small(pretrained=True), "Baseline_dslr.pth", dslr_vl, dslr_vl, False, False],
    # [models.mobilenet_v3_small(pretrained=True), "Baseline_webcam.pth", webcam_vl, webcam_vl, False, False],
]

# Run the evaluation and save results to CSV
evaluate_models(lst)

Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Mod Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=31, bias=True)
)
Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Mod Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=31, bias=True)
)
Original Classifier: Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear

<ipython-input-13-59cc2fc08358>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH))


[(Inference || test loss: 1.8786544799804688] [accuracy_test: 52.800000000000004 %]


100%|██████████| 6/6 [00:04<00:00,  1.47it/s]
<ipython-input-3-ad3c33898564>:295: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  plt.scatter(


Saving t-SNE to ./output/tSNE_SO_amazon_to_dslr.png
epoch 0 accuracy: 68.98148345947266 A-dist: 0.7592592239379883
epoch 1 accuracy: 85.64814758300781 A-dist: 1.4259259700775146
epoch 2 accuracy: 82.87036895751953 A-dist: 1.314814805984497
epoch 3 accuracy: 86.5740737915039 A-dist: 1.4629628658294678
epoch 4 accuracy: 87.96295928955078 A-dist: 1.5185184478759766
epoch 5 accuracy: 82.87036895751953 A-dist: 1.314814805984497
epoch 6 accuracy: 82.87036895751953 A-dist: 1.314814805984497
epoch 7 accuracy: 82.40740966796875 A-dist: 1.2962963581085205
epoch 8 accuracy: 83.33333587646484 A-dist: 1.3333334922790527
epoch 9 accuracy: 80.55555725097656 A-dist: 1.2222223281860352
A-distance = 1.2222223281860352
Confusion Matrix saved to ./output/CM_SO_amazon_to_dslr.png
Precision: 0.6242962369376743, Recall: 0.5335120643431636, F-score: 0.5280227384225699
NMI score: 0.6606503034165998, RI score: 0.31583403590716197


...

Saving t-SNE to ./output/tSNE_DANN(Teacher)_webcam_to_amazon.png
epoch 0 accuracy: 76.24309539794922 A-dist: 1.0497238636016846
epoch 1 accuracy: 19.889503479003906 A-dist: -1.2044198513031006
epoch 2 accuracy: 79.55801391601562 A-dist: 1.1823205947875977
epoch 3 accuracy: 22.651933670043945 A-dist: -1.0939226150512695
epoch 4 accuracy: 22.651933670043945 A-dist: -1.0939226150512695
epoch 5 accuracy: 79.55801391601562 A-dist: 1.1823205947875977
epoch 6 accuracy: 59.66850662231445 A-dist: 0.38674020767211914
epoch 7 accuracy: 79.55801391601562 A-dist: 1.1823205947875977
epoch 8 accuracy: 85.08287048339844 A-dist: 1.4033148288726807
epoch 9 accuracy: 82.8729248046875 A-dist: 1.3149170875549316
A-distance = 1.3149170875549316
Confusion Matrix saved to ./output/CM_DANN(Teacher)_webcam_to_amazon.png
Precision: 0.29167377219059387, Recall: 0.2581560283687943, F-score: 0.24234775579760567
NMI score: 0.424297878360507, RI score: 0.13915187421872927


<ipython-input-13-59cc2fc08358>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH))


[(Inference || test loss: 0.8799011707305908] [accuracy_test: 87.22999999999999 %]


100%|██████████| 2/2 [00:01<00:00,  1.48it/s]


epoch 0 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 1 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 2 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 3 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 4 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 5 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 6 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 7 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 8 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
epoch 9 accuracy: 53.846153259277344 A-dist: 0.15384602546691895
A-distance = 0.15384602546691895
Confusion Matrix saved to ./output/CM_DANN(Teacher)_webcam_to_dslr.png
Precision: 0.8639333333333333, Recall: 0.872, F-score: 0.855912087912088
NMI score: 0.9371508395606333, RI score: 0.838049973879028
Results saved to evaluation_results.csv


# Utils

In [ ]:
%matplotlib inline
plt.show()

In [ ]:
import shutil

# Delete the "/content/cp" folder and its contents
shutil.rmtree('/content/output')

print("Folder '/content/output' deleted successfully.")

Folder '/content/output' deleted successfully.


In [ ]:
zip_directory("/content/output", "All_Graph.zip")

In [ ]:
def plot_comparison(file1_data, file2_data, file1_name="File 1", file2_name="File 2", save_name="file1_vs_file2.png"):
    """
    This function plots three graphs (Train Loss, Source Accuracy, and Target Accuracy) comparing two datasets.
    It then saves the figure with the provided filename.

    Args:
    - file1_data (dict): Dictionary with keys 'train_loss', 'train_src_acc', 'train_tar_acc' for the first dataset.
    - file2_data (dict): Dictionary with keys 'train_loss', 'train_src_acc', 'train_tar_acc' for the second dataset.
    - file1_name (str): Name to label the first dataset on the plots.
    - file2_name (str): Name to label the second dataset on the plots.
    - save_name (str): Filename to save the generated plot image.
    """
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Plot 1: Train Loss
    axs[0].plot(file1_data["train_loss"], label=f'{file1_name} Loss')
    axs[0].plot(file2_data["train_loss"], label=f'{file2_name} Loss')
    axs[0].set_title('Train Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    # Plot 2: Source Accuracy
    axs[1].plot(file1_data["train_src_acc"], label=f'{file1_name} Source Accuracy')
    axs[1].plot(file2_data["train_src_acc"], label=f'{file2_name} Source Accuracy')
    axs[1].set_title('Source Domain Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()

    # Plot 3: Target Accuracy
    axs[2].plot(file1_data["train_tar_acc"], label=f'{file1_name} Target Accuracy')
    axs[2].plot(file2_data["train_tar_acc"], label=f'{file2_name} Target Accuracy')
    axs[2].set_title('Target Domain Accuracy')
    axs[2].set_xlabel('Epochs')
    axs[2].set_ylabel('Accuracy')
    axs[2].legend()

    # Save the plots to a file
    plt.tight_layout()
    plt.savefig(save_name)

    # Show the plots
    plt.show()


# load_training_logs(filename)
log_pairs = [
    ["LOG_DANN_amazon_to_dslr", "LOG_SO_amazon_to_dslr"],
    ["LOG_DANN_amazon_to_webcam", "LOG_SO_amazon_to_webcam"],
    ["LOG_DANN_dslr_to_amazon", "LOG_SO_dslr_to_amazon"],
    ["LOG_DANN_dslr_to_webcam", "LOG_SO_dslr_to_webcam"],
    ["LOG_DANN_webcam_to_amazon", "LOG_SO_webcam_to_amazon"],
    ["LOG_DANN_webcam_to_dslr", "LOG_SO_webcam_to_dslr"]
]
for so, dann in log_pairs:
    file1_data = load_training_logs(so)
    file2_data = load_training_logs(dann)
    plot_comparison(file1_data, file2_data, file1_name=f"{so}", file2_name=f"{dann}", save_name=f"{so}_vs_{dann}.png")

Training logs loaded from LOG_DANN_amazon_to_dslr
Training logs loaded from LOG_SO_amazon_to_dslr
Training logs loaded from LOG_DANN_amazon_to_webcam
Training logs loaded from LOG_SO_amazon_to_webcam
Training logs loaded from LOG_DANN_dslr_to_amazon
Training logs loaded from LOG_SO_dslr_to_amazon
Training logs loaded from LOG_DANN_dslr_to_webcam
Training logs loaded from LOG_SO_dslr_to_webcam
Training logs loaded from LOG_DANN_webcam_to_amazon
Training logs loaded from LOG_SO_webcam_to_amazon
Training logs loaded from LOG_DANN_webcam_to_dslr
Training logs loaded from LOG_SO_webcam_to_dslr
